# USER INPUT

**Paste the Excel Line in the Cell Below!**

In [118]:
params = {
"event_id": "nc72282711",
"geography": "tract",
"intensity_metric": "min",
"utility_loss_rate": 0.20
}

# CODE BASE




## 1. Setting Google Environment

In [119]:
EVENTID = params["event_id"]

In [120]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd


In [121]:
'''# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')'''

"# Mount the Google Drive\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [122]:
'''# Setting System to Import Packages
import sys
sys.path.append('/content/drive/MyDrive/capstone/WorkingScripts')
parent_dir = '/content/drive/MyDrive/capstone/'''

"# Setting System to Import Packages\nimport sys\nsys.path.append('/content/drive/MyDrive/capstone/WorkingScripts')\nparent_dir = '/content/drive/MyDrive/capstone/"

In [123]:
parent_dir = os.path.dirname(os.getcwd())

In [124]:
# Install Required Packages
import environment_setting as es # Replace my_module with the actual name of your module

es.setting_env(parent_dir)

Environment is set up successfully.


# ANALYSIS


## 1. Get Shakemap for the related earthquake from the USGS database.

In [125]:
# Running o1
import o1_getshakemap as getshakemap

feed_url = getshakemap.FEEDURL.format(EVENTID)
jdict = getshakemap.fetch_earthquake_data(feed_url=feed_url)

event = getshakemap.retrieve_event_data(jdict)
getshakemap.download_and_extract_shakemap(event)

Extracted ShakeMap for event nc72282711 to /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711
Success! Downloaded ShakeMap for event nc72282711...


'/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711'

## 2. Get Census Tract data then intersect it with the earthquake's region.

In [126]:
def get_state_list(parent_dir, EVENTID):
    pga = os.path.join(parent_dir, 'ShakeMaps', EVENTID, 'pga.shp')
    pga_gpd = gpd.read_file(pga)

    state_shp = os.path.join(parent_dir, 'Data', 'tl_2024_us_state', 'tl_2024_us_state.shp')
    state_gpd = gpd.read_file(state_shp)

    state_gpd = state_gpd.to_crs(pga_gpd.crs)
    pga_gpd = pga_gpd.loc[pga_gpd['PARAMVALUE']>=0.1]
    clipped_pga = gpd.sjoin(pga_gpd, state_gpd, how="inner", predicate="intersects")
    state_df = clipped_pga[['GEOID', 'PARAMVALUE']].groupby('GEOID').count().reset_index()
    state_list = state_df['GEOID'].tolist()
    return state_list



In [127]:
import o2_download_census_new as download_census
state_list = get_state_list(parent_dir, EVENTID)
parent_dir = os.path.dirname(os.getcwd())
# on google colab
# parent_dir = '/content/drive/MyDrive/capstone_v2'
# download census of specific states
download_census.download_census(state_list, parent_dir)

attempting to link to https://www2.census.gov/geo/tiger/TIGER2024/TRACT/
parsing...
Found 1 ZIP files. Starting download...
Download Census Data to /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/census_shp
Extracting ZIP files...
Extracted: tl_2024_16_tract.zip
Extracted: tl_2024_06_tract.zip
Merging shapefiles...
Reading /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/extracted_census_shp/tl_2024_06_tract.shp...
Reading /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/extracted_census_shp/tl_2024_16_tract.shp...
Nationwide Census Tracts saved to: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/merged_shapefile/Nationwide_Tracts.gpkg


,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,001,442700,06001442700,1400000US06001442700,4427,Census Tract 4427,G5020,S,1234016,0,+37.5371513,-122.0081095,"POLYGON ((-122.01721 37.53932, -122.01719 37.5..."
1,06,001,442800,06001442800,1400000US06001442800,4428,Census Tract 4428,G5020,S,1278646,0,+37.5293619,-121.9931002,"POLYGON ((-122.0023 37.52984, -122.00224 37.52..."
2,06,037,204920,06037204920,1400000US06037204920,2049.20,Census Tract 2049.20,G5020,S,909972,0,+34.0175004,-118.1974975,"POLYGON ((-118.20284 34.01966, -118.20283 34.0..."
3,06,037,205110,06037205110,1400000US06037205110,2051.10,Census Tract 2051.10,G5020,S,286962,0,+34.0245059,-118.2142985,"POLYGON ((-118.21964 34.02628, -118.21945 34.0..."
4,06,037,205120,06037205120,1400000US06037205120,2051.20,Census Tract 2051.20,G5020,S,1466242,0,+34.0187542,-118.2117951,"POLYGON ((-118.22023 34.02056, -118.22018 34.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9580,16,031,950300,16031950300,1400000US16031950300,9503,Census Tract 9503,G5020,S,54828209,640084,+42.4805097,-113.7526916,"POLYGON ((-113.79382 42.45163, -113.79376 42.4..."
9581,16,031,950200,16031950200,1400000US16031950200,9502,Census Tract 9502,G5020,S,163331895,5666958,+42.5409128,-113.6227583,"POLYGON ((-113.72492 42.43151, -113.72492 42.4..."
9582,16,031,950100,16031950100,1400000US16031950100,9501,Census Tract 9501,G5020,S,4213342323,24740144,+42.2900770,-113.3791677,"POLYGON ((-113.84026 42.2099, -113.84003 42.21..."
9583,16,007,950200,16007950200,1400000US16007950200,9502,Census Tract 9502,G5020,S,1305163495,102838367,+42.3095610,-111.4817532,"POLYGON ((-111.6349 42.57004, -111.63488 42.57..."


In [128]:
# Running o2 census intersect
import o2_census_intersect as census

event_dir = os.path.join(parent_dir, 'ShakeMaps', EVENTID)

census.shakemap_into_census_geo(event_dir, parent_dir)

# Update with the actual path
GPKG_PATH = os.path.join(event_dir, "eqmodel_outputs.gpkg")


Census tracts shapefile already exists /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/merged_shapefile/Nationwide_Tracts.gpkg
Creating GeoPackage: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711/eqmodel_outputs.gpkg


/opt/anaconda3/envs/capstone/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


Saved shakemap_tractclip_pga (clipped ShakeMap to tracts) to /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711/eqmodel_outputs.gpkg
=======================  pga clipped ====
      AREA  PERIMETER  PGAPOL_  PGAPOL_ID  GRID_CODE  PARAMVALUE STATEFP  \
0      0.0        0.0        0          0          0        0.01      06   
1      0.0        0.0        0          0          0        0.01      06   
2      0.0        0.0        0          0          0        0.01      06   
3      0.0        0.0        0          0          0        0.01      06   
4      0.0        0.0        0          0          0        0.01      06   
...    ...        ...      ...        ...        ...         ...     ...   
3968   0.0        0.0       29         29          0        0.58      06   
3969   0.0        0.0       29         29          0        0.58      06   
3970   0.0        0.0       29         29     

## 3. Get the number of building in all earthquake-affected census tract.

In [129]:
# Running o3 clip building
import o3_clip_eventdata_buildingstocks as bs
o3output = bs.building_clip_analysis(parent_dir, EVENTID)

1. Reading event data for event ID: nc72282711
2. Reading building count data...
3. Reading building stock data...
Building stock data exists at /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Tables/Building_Percentages_Per_Tract_ALLSTATES.csv
4. Merging building count and building stock data...
5. Merging event data with building data...
            GEOID  max_intensity  min_intensity  mean_intensity  \
0     06001400100           0.02           0.02            0.02   
1     06001400200           0.02           0.02            0.02   
2     06001400300           0.02           0.02            0.02   
3     06001400400           0.02           0.02            0.02   
4     06001400500           0.02           0.02            0.02   
...           ...            ...            ...             ...   
2977  06115040902           0.01           0.01            0.01   
2978  06115041001           0.01           0.

## 4. Estimate the building damage from the earthquake.

In [130]:
# Run o4
import o4_TractLevel_DamageAssessmentModel as o4
o4_output = o4.build_damage_estimates(o3output, parent_dir)

In [131]:
o4_output


,GEOID,max_intensity,min_intensity,mean_intensity,geometry,Total_Num_Building,Total_Num_Building_Slight,Total_Num_Building_Moderate,Total_Num_Building_Extensive,Total_Num_Building_Complete
0,06001400100,0.02,0.02,0.02,"MULTIPOLYGON (((-122.24691 37.88536, -122.2466...",1298.0,0.050514,1.045180e-06,6.984460e-12,3.361176e-18
1,06001400200,0.02,0.02,0.02,"MULTIPOLYGON (((-122.25792 37.84261, -122.2577...",696.0,0.033581,1.912345e-06,3.798276e-11,2.482994e-17
2,06001400300,0.02,0.02,0.02,"MULTIPOLYGON (((-122.26563 37.83764, -122.2655...",1629.0,0.084944,5.205597e-06,8.573605e-11,5.436552e-17
3,06001400400,0.02,0.02,0.02,"MULTIPOLYGON (((-122.26183 37.84162, -122.2618...",1228.0,0.060099,3.968547e-06,7.735657e-11,5.033211e-17
4,06001400500,0.02,0.02,0.02,"MULTIPOLYGON (((-122.26951 37.84858, -122.2693...",1018.0,0.054968,4.015263e-06,7.736307e-11,5.033211e-17
...,...,...,...,...,...,...,...,...,...,...
2977,06115040902,0.01,0.01,0.01,"MULTIPOLYGON (((-121.47722 39.13334, -121.4770...",759.0,0.000702,1.539085e-09,9.440610e-17,8.301358e-26
2978,06115041001,0.01,0.01,0.01,"MULTIPOLYGON (((-121.63637 39.24608, -121.6362...",2836.0,0.012889,2.383796e-08,1.370821e-15,1.752558e-24
2979,06115041002,0.01,0.01,0.01,"MULTIPOLYGON (((-121.46948 39.33348, -121.469 ...",2824.0,0.012850,2.378486e-08,1.369195e-15,1.750766e-24
2980,06115041101,0.01,0.01,0.01,"MULTIPOLYGON (((-121.37257 39.37358, -121.3725...",1958.0,0.008884,1.640822e-08,9.424219e-16,1.204640e-24


## 5. Incorporate the Building Habitability Index and Social Vulnerability Index to estimate seeking shelters population.

In [132]:
import pandas as pd
import numpy as np
o4out = o4_output.copy()

In [133]:
o4out['GEOID'] = o4out['GEOID'].astype(str)
o4out['GEOID'] = np.where(o4out['GEOID'].str.len() == 11, o4out['GEOID'], o4out['GEOID'].str.zfill(11))
o4out['GEOID'].unique()

array(['06001400100', '06001400200', '06001400300', ..., '06115041002',
       '06115041101', '06115041102'], dtype=object)

In [134]:
from o5_bhi import process_bhi
from o5_svi_module import process_svi
import os
parent_dir = os.path.dirname(os.getcwd())
# ================================================
# o5 - Implement BHI
# ================================================
df = process_bhi(o4out, parent_dir)


df["population"] = df["population"].astype(int)
df["shelter_seeking_low"] = df["BHI_factor_low"]*df["population"]
df["shelter_seeking_high"] = df["BHI_factor_high"]*df["population"]
cols = ["GEOID", "max_intensity", "population", "Total_Num_Building", "risk_level",
        "BHI_factor_low", "BHI_factor_high",
        "shelter_seeking_low", "shelter_seeking_high",
        "Total_Num_Building_Slight", "Total_Num_Building_Moderate", 
            "Total_Num_Building_Extensive", "Total_Num_Building_Complete"]
df = df[cols]
df["GEOID"] = df["GEOID"].astype(str)


# ================================================
# o6 - Download SVI data 
# ================================================
# download SVI (optional)
# process SVI

# apply SVI 
svi = process_svi(parent_dir)
svi["FIPS"] = svi["FIPS"].astype(str)
svi["FIPS"] = np.where(svi["FIPS"].str.len() == 11, svi["FIPS"], svi["FIPS"].str.zfill(11))


df = df.merge(svi, left_on = "GEOID", right_on="FIPS")

# ================================================
# o7 - Combine SVI and BHI, Format Output Data
# ================================================

df["shelter_seeking_low"] = df["shelter_seeking_low"]*df["SVI_Value_Mapped"] 
df["shelter_seeking_high"] = df["shelter_seeking_high"]*df["SVI_Value_Mapped"]

df = df.drop(columns=["FIPS"])

print(df["shelter_seeking_low"].sum())
print(df["shelter_seeking_low"].sum()/df["population"].sum())

df = df.sort_values(by=["shelter_seeking_low"], ascending=False)

print(df.head(50))

/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/WorkingScripts/o5_bhi.py:65: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  pop_data = pd.read_csv(pop_path)


63.06892915780134
4.842266212753868e-06
          GEOID  max_intensity  population  Total_Num_Building risk_level  \
7   06055200501           0.54        5159              1364.0        low   
13  06055200504           0.50        6577              1483.0        low   
32  06095251802           0.34        3132               721.0        low   
14  06055200301           0.50        5602              1433.0        low   
35  06055201004           0.32        6836              1615.0        low   
9   06055200202           0.54        2479               697.0        low   
5   06055200804           0.58        5535              1036.0        low   
18  06055200601           0.48        4892              1461.0        low   
48  06095250701           0.28        3473              1145.0        low   
6   06055200201           0.56        3259              1149.0        low   
3   06055200705           0.58        2845               783.0        low   
11  06055200704           0.54      

## 6. Saving the Final Output

In [135]:
save_path = os.path.join(parent_dir, f"Data/final_output_new_{EVENTID}.csv")
df.to_csv(save_path, index=False)

save_excel_path = os.path.join(parent_dir, f"Data/final_output_new_{EVENTID}.xlsx")
df.to_excel(save_excel_path, index=False)